In [1]:
import cv2
import os

path_of_data=r'D:\ML_PROJECT\End_sem\image'
cat=os.listdir(path_of_data)
my_labels=[]
l=len(cat)
for i in range(l):
    my_labels.append(i)
result=zip(cat,my_labels)
dict_of_label=dict(result) #empty dictionary

print(dict_of_label)
print(cat)
print(my_labels)

{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]


In [ ]:
my_data=[]
target_list=[]
size_image=100


for c in cat:
    path_folder=os.path.join(path_of_data,c)
    
    names_of_img=os.listdir(path_folder)
        
    for k in names_of_img:
        path_image=os.path.join(path_folder,k)
        img=cv2.imread(path_image)
        
        #Coverting the image into gray scale
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
        
        #resizing the gray scale into 50x50, since we need a fixed common size for all the images in the dataset
        resized_image=cv2.resize(gray,(size_image,size_image))
        
        #appending the image and the label(categorized) into the list (dataset)
        my_data.append(resized_image)
        target_list.append(dict_of_label[c])
        

       

In [ ]:
import numpy as np
from keras.utils import np_utils

my_data=np.array(my_data)
max_pixel=my_data.max()
my_data=my_data/max_pixel
no_of_rows=my_data.shape[0]
my_data=np.reshape(my_data,(no_of_rows,size_image,size_image,1))
target_list=np.array(target_list)

new_target_list=np_utils.to_categorical(target_list)


In [ ]:
# saving the data in npy format
np.save('prepared_data',my_data)
np.save('target_val',new_target_list)

In [ ]:
import numpy as np

#loading the save numpy arrays in the previous code

data_prepared=np.load('prepared_data.npy')
target_value=np.load('target_val.npy')



In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model_used=Sequential()

model_used.add(Conv2D(200,(3,3),input_shape=data_prepared.shape[1:]))
model_used.add(Activation('relu'))
model_used.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model_used.add(Conv2D(100,(3,3)))
model_used.add(Activation('relu'))
model_used.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model_used.add(Flatten())
model_used.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model_used.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model_used.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model_used.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split

training_data,test_data,training_target,test_target=train_test_split(data_prepared,target_value,test_size=0.2,random_state=10,stratify=target_value)

In [ ]:
compilation = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
final_model=model_used.fit(training_data,training_target,epochs=10,callbacks=[compilation],validation_data=(test_data,test_target))

In [ ]:
from matplotlib import pyplot as plt

plt.plot(final_model.history['loss'],'r',label='Training Loss')
plt.plot(final_model.history['val_loss'],'g',label='Validation Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(final_model.history['accuracy'],'r',label='Training accuracy')
plt.plot(final_model.history['val_accuracy'],label='Validation accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
print(model_used.evaluate(test_data,test_target))

In [ ]:
from keras.models import load_model
import cv2
import numpy as np

In [ ]:
my_model = load_model('model-005.model')

face_dtctr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

video=cv2.VideoCapture(0)

labels_face={0:'NO MASK',1:'WITH MASK'}
color_dict={1:(0,255,0),0:(0,0,255)}

In [ ]:
while(True):

    r,image1=video.read()
    gray=cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
    faces=face_dtctr.detectMultiScale(gray,1.08,5)  

    for (x,y,w,h) in faces:
    
        face_image=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_image,(100,100))
        resized_new=resized/255.0
        final_image=np.reshape(resized_new,(1,100,100,1))
        res=my_model.predict(final_image)
        label1=np.argmax(res,axis=1)
        label=label1[0]
        color_given = (255, 255, 255)
        cv2.rectangle(image1,(x,y),(x+w,y+h),color_dict[label],3)
        cv2.rectangle(image1,(x,y-48),(x+w,y),color_given,-1)
        cv2.putText(image1, labels_face[label], (x, y-9),cv2.FONT_HERSHEY_SIMPLEX,0.9,(128,0,0),2)
        
        
    cv2.imshow('LIVE',image1)
    key_used=cv2.waitKey(1)
    
    if(key_used==27):
        break
        
cv2.destroyAllWindows()
video.release()